In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.speakernet as speakernet
import tensorflow as tf


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.speakernet_featurizer_config
featurizer = malaya_speech.featurization.SpeakerNetFeaturizer(config)

In [5]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [6]:
%%time
featurizer.vectorize(y).shape

CPU times: user 16.8 ms, sys: 4.67 ms, total: 21.4 ms
Wall time: 13.1 ms


(564, 64)

In [9]:
v = tf.placeholder(tf.float32, [None, None, 64])
v_length = tf.placeholder(tf.int32, [None])

In [11]:
model = speakernet.Model(v, v_length)
model.logits


Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use `tf.keras.layers.SeparableConv1D` instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.


<tf.Tensor 'Relu_1:0' shape=(?, 512) dtype=float32>

In [12]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [13]:
variables = [v for v in tf.get_collection('variables')]
variables

[<tf.Variable 'w2l_encoder/conv11/res/kernel:0' shape=(1, 64, 512) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/res_bn/gamma:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/res_bn/beta:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/res_bn/moving_mean:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/res_bn/moving_variance:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/depthwise_kernel:0' shape=(3, 64, 1) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/pointwise_kernel:0' shape=(1, 64, 512) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/bn/gamma:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/bn/beta:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/bn/moving_mean:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv11/bn/moving_variance:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'w2l_encoder/conv21/depthwise_kernel:

In [18]:
padded, l = malaya_speech.padding.sequence_nd([featurizer.vectorize(y), featurizer.vectorize(y1)], 
                                              dim = 0, return_len = True)

In [20]:
sess.run(model.logits, feed_dict = {v: padded, v_length: l}).shape

(2, 512)